In [29]:

from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [31]:
# incarcam liniie din fisier
with open("sonnets.txt", "r") as file:
    sonnets_text = file.read()

# structurez sonetele incarcate ca sir, dupa dublu spatiu
sonnets = [sonnet.strip() for sonnet in sonnets_text.split('\n\n') if sonnet.strip()]
sonnets = sonnets[:20]

# creez promturi care sunt primul vers a fiecarui sonet si slavez originalul
prompts = [sonnet.split('\n')[0] for sonnet in sonnets]
original_sonnets = sonnets

# afisez promturile
print("\n=== Prompts (First Lines of Each Sonnet) ===")
for i, prompt in enumerate(prompts, 1):
    print(f"Prompt {i}: {prompt}")
print("\n")



=== Prompts (First Lines of Each Sonnet) ===
Prompt 1: FROM fairest creatures we desire increase,
Prompt 2: When forty winters shall beseige thy brow,
Prompt 3: Look in thy glass, and tell the face thou viewest
Prompt 4: Unthrifty loveliness, why dost thou spend
Prompt 5: Those hours, that with gentle work did frame
Prompt 6: Then let not winter's ragged hand deface
Prompt 7: Lo! in the orient when the gracious light
Prompt 8: Music to hear, why hear'st thou music sadly?
Prompt 9: Is it for fear to wet a widow's eye
Prompt 10: For shame! deny that thou bear'st love to any,
Prompt 11: As fast as thou shalt wane, so fast thou growest
Prompt 12: When I do count the clock that tells the time,
Prompt 13: O, that you were yourself! but, love, you are
Prompt 14: Not from the stars do I my judgment pluck;
Prompt 15: When I consider every thing that grows
Prompt 16: But wherefore do not you a mightier way
Prompt 17: Who will believe my verse in time to come,
Prompt 18: Shall I compare thee to 

In [39]:
# incarc modelul the gpt-2 si tokenizerul
model_name = "gpt2"
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name).eval()
except Exception as e:
    print(f"Error loading model: {e}")
    print("Ensure PyTorch is installed: pip install torch")
    exit()


In [40]:
# ma asigur ca attention maskul are un padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [45]:
# functii the generare a versurilor si attention mark

def generate_verse(prompt, max_length=200, temperature=0.8, top_k=40, top_p=0.9):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    )
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=2
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [46]:
generated_sonnets = []
bleu_scores = []
smoothie = SmoothingFunction().method4

for prompt, original in zip(prompts, original_sonnets):
    generated_verse = generate_verse(prompt)
    generated_sonnets.append(generated_verse)

    #calculeaza scorul bleu
    reference = [original.split()]
    candidate = generated_verse.split()
    score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    bleu_scores.append(score)


In [47]:
# arata diferenta intre sonetul original si generat cu Bleu score
print("=== Final Comparison: Original vs. Generated Sonnets ===")
for i, (original, generated, score) in enumerate(zip(original_sonnets, generated_sonnets, bleu_scores), 1):
    print(f"\nSonnet {i}:")
    print("Original:")
    print(original.replace('\n', '\n  '))
    print("\nGenerated:")
    print(generated.replace('\n', '\n  '))
    print(f"\nBLEU Score: {score:.4f}")
print(f"\nAverage BLEU: {sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0:.4f}")

=== Final Comparison: Original vs. Generated Sonnets ===

Sonnet 1:
Original:
FROM fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou, contracted to thine own bright eyes,
  Feed'st thy light'st flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thyself thy foe, to thy sweet self too cruel.
  Thou that art now the world's fresh ornament
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content
  And, tender churl, makest waste in niggarding.
  Pity the world, or else this glutton be,
  To eat the world's due, by the grave and thee.

Generated:
FROM fairest creatures we desire increase, and we will not be able to do so without the help of the people.
  
  The people will be the first to see that we are not only a nation, but a people, that is, a country. We will have a government that will protect us from the dan